In [3]:
using Pkg
Pkg.activate("Phase2_projet")
Pkg.add(["LinearAlgebra", "SparseArrays", "Krylov", "BenchmarkTools", "SuiteSparseMatrixCollection", "MatrixMarket"])
using LinearAlgebra, SparseArrays, Krylov, BenchmarkTools, SuiteSparseMatrixCollection, MatrixMarket, Random

  Activating project at `c:\Users\ulric\Projet_MTH8211\Phase2_projet`
   Resolving package versions...
  No Changes to `C:\Users\ulric\Projet_MTH8211\Phase2_projet\Project.toml`
  No Changes to `C:\Users\ulric\Projet_MTH8211\Phase2_projet\Manifest.toml`


In [4]:
println("Nombre de threads actuellement activés : ", Threads.nthreads())

Nombre de threads actuellement activés : 1


In [5]:
function badly_conditioned_rectangular_matrix(m, n, kappa)
    U, _ = qr(randn(m, n))
    V, _ = qr(randn(n, n))
    s = range(1.0, 1.0/kappa, length=n)
    S = Diagonal(s)
    A = U * S * V'
    return Matrix(A)
end

badly_conditioned_rectangular_matrix (generic function with 1 method)

In [6]:
function badly_conditioned_underdetermined_matrix(m, n, kappa)
    U, _ = qr(randn(n, m))
    V, _ = qr(randn(m, m))
    s = range(1.0, 1.0 / kappa, length=m)
    S = Diagonal(s)
    A_tall = U * S * V'
    return Matrix(A_tall')[1:m, 1:n]
end

badly_conditioned_underdetermined_matrix (generic function with 1 method)

In [7]:
function lsrn_lsqr(A, b; gamma=2.0, tol=1e-10, itmax=2000)
    m, n = size(A)
    s = ceil(Int, gamma * n) 

    G = randn(s, m)

    Ã = G * A

    Ũ, Σ̃, Ṽ = svd(Ã; full=false)
    r = sum(Σ̃.> 1e-12)

    Σinv = Diagonal(1.0 ./ Σ̃[1:r])
    V_r = Ṽ[:,1:r]
    N = V_r * Σinv

    AN = A * N
    ŷ, histo = lsqr(AN, b; atol=tol, btol=tol, itmax=itmax, history=true)

    x̂ = N * ŷ
    return x̂, histo
end

lsrn_lsqr (generic function with 1 method)

In [8]:
function lsrn_lsqr_underdetermined(A, b; gamma=2.0, tol=1e-10, itmax=2000)
    m, n = size(A)
    s = ceil(Int, gamma * m)
    G = randn(n, s)
    Ã = A * G

    Ũ, Σ̃, Ṽ = svd(Ã; full=false)
    r = sum(Σ̃ .> 1e-12)
    U_r = Ũ[:,1:r]
    Σ_r = Σ̃[1:r]

    Σinv = Diagonal(1.0 ./ Σ_r)
    M = U_r * Σinv      

    Mt = M'
    At_pre = Mt * A    
    bt_pre = Mt * b     

    x̂, histo = lsqr(At_pre, bt_pre; atol=tol, btol=tol, itmax=itmax, history=true)

    return x̂, histo
end

lsrn_lsqr_underdetermined (generic function with 1 method)

Validation du bon focntionnement de l'algo

In [35]:
m = 10
n = 10
kappa = 2
A = badly_conditioned_rectangular_matrix(m, n, kappa)
x = randn(n)
b = A * x
solve_QR(A, b) = qr(A)\ b
x_qr = solve_QR(A, b)
x_lsrn_lsqr, histo = lsrn_lsqr(A, b; gamma=2.0, tol=1e-10, itmax=2000) 
norm(x_qr - x_lsrn_lsqr)

9.945618297179095e-14

Création d'une matrice fortement rectangulaire mal conditionnée test

In [21]:
m = 10000
n = 1000
kappa = 1e10
A = badly_conditioned_rectangular_matrix(m, n, kappa)
x = randn(n)
b = A * x
println("Conditionnement de A :", cond(A))

Conditionnement de A :9.999999777081144e9


Vérification de l'amélioration du conditionnement

In [22]:
gamma = 2.0
m, n = size(A)
s = ceil(Int, gamma * n)
G = randn(s, m)
Ã = G * A
Ũ, Σ̃, Ṽ = svd(Ã; full=false)
r = sum(Σ̃.> 1e-12)
Σinv = Diagonal(1.0 ./ Σ̃[1:r])
V_r = Ṽ[:,1:r]
N = V_r * Σinv
AN = A * N
println("Conditionnement de A :", cond(A))
println("Conditionnement de A prec :", cond(AN))

Conditionnement de A :9.999999777081144e9
Conditionnement de A prec :5.76888460257047


In [ ]:
BLAS.set_num_threads(16)
BLAS.get_num_threads()

In [10]:
@benchmark lsrn_lsqr(A, b; gamma=2.0, tol=1e-10, itmax=2000) 

BenchmarkTools.Trial: 6 samples with 1 evaluation per sample.
 Range (min … max):  855.905 ms …    1.129 s  ┊ GC (min … max): 2.63% … 10.34%
 Time  (median):     903.858 ms               ┊ GC (median):    2.01%
 Time  (mean ± σ):   940.774 ms ± 101.650 ms  ┊ GC (mean ± σ):  3.48% ±  3.66%

  █   ██          █         █                                 █  
  █▁▁▁██▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  856 ms           Histogram: frequency by time          1.13 s <

 Memory estimate: 328.40 MiB, allocs estimate: 79.

In [16]:
@benchmark lsqr(A, b; atol=1e-10, btol=1e-10, itmax=2000, history=true)

BenchmarkTools.Trial: 2 samples with 1 evaluation per sample.
 Range (min … max):  2.894 s …   2.953 s  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.924 s              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.924 s ± 42.034 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                                       █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  2.89 s         Histogram: frequency by time        2.95 s <

 Memory estimate: 372.52 KiB, allocs estimate: 42.

In [19]:
# LSQR 
println("\n--- LSQR sans préconditionneur ---")
res1, hist1 = lsqr(A, b; atol=1e-10, btol=1e-10, itmax=2000, history=true)
println("Itérations LSQR (sans préc)      : ", hist1.niter)
println("Résidu final LSQR (sans préc)    : ", norm(A * res1 - b) / norm(b))

#lsqr-lsrn
println("\n--- LSQR avec préconditionneur LSRN ---")
x_prec, hist2 = lsrn_lsqr(A, b; gamma=2.0, tol=1e-10, itmax=2000)
println("Itérations LSQR (LSRN)           : ", hist2.niter)
println("Résidu final LSQR (LSRN)         : ", norm(A * x_prec - b) / norm(b))

println("\n--- Résumé du nombre d'itérations ---")
println("LSQR sans préc           : $(hist1.niter)")
println("LSQR avec préc LSRN      : $(hist2.niter)")



--- LSQR sans préconditionneur ---
Itérations LSQR (sans préc)      : 1499
Résidu final LSQR (sans préc)    : 4.1583028761218144e-7

--- LSQR avec préconditionneur LSRN ---
Itérations LSQR (LSRN)           : 42
Résidu final LSQR (LSRN)         : 2.224226840460454e-7

--- Résumé du nombre d'itérations ---
LSQR sans préc           : 1499
LSQR avec préc LSRN      : 42


In [14]:
m = 1000   
n = 10000  
kappa = 1e9
A = badly_conditioned_underdetermined_matrix(m, n, kappa)
x_exact = randn(n)
b = A * x_exact

println("Conditionnement estimé (A*A') :", cond(A * A'))

# LSQR direct 
println("\n--- LSQR standard (sous-déterminé) ---")
res1, hist1 = lsqr(A, b; atol=1e-10, btol=1e-10, itmax=2000, history=true)
println("Itérations LSQR : ", hist1.niter)
println("Résidu final LSQR : ", norm(A * res1 - b) / norm(b))

# LSRN + LSQR
println("\n--- LSRN (LSQR, sous-déterminé) ---")
x_lsrn_lsqr, hist2 = lsrn_lsqr_underdetermined(A, b; gamma=2.0, tol=1e-10, itmax=2000)
println("Itérations LSQR (LSRN) : ", hist2.niter)
println("Résidu LSRN (LSQR) : ", norm(A * x_lsrn_lsqr - b) / norm(b))



println("\n--- Résumé du nombre d'itérations ---")
println("LSQR standard         : $(hist1.niter)")
println("LSRN LSQR             : $(hist2.niter)")

Conditionnement estimé (A*A') :2.846313755985417e16

--- LSQR standard (sous-déterminé) ---
Itérations LSQR : 1366
Résidu final LSQR : 6.811089381322153e-7

--- LSRN (LSQR, sous-déterminé) ---
Itérations LSQR (LSRN) : 43
Résidu LSRN (LSQR) : 2.1488075681210543e-7

--- Résumé du nombre d'itérations ---
LSQR standard         : 1366
LSRN LSQR             : 43


### CODE NON TERMINÉ OU NON UTILISÉ

In [17]:
function lsrn_lsmr(A, b; gamma=2.0, tol=1e-10, itmax=2000)
    m, n = size(A)
    s = ceil(Int, gamma * n) 

    G = randn(s, m)

    Ã = G * A

    Ũ, Σ̃, Ṽ = svd(Ã; full=false)
    r = sum(Σ̃.> 1e-12)

    Σinv = Diagonal(1.0 ./ Σ̃[1:r])
    V_r = Ṽ[:,1:r]
    N = V_r * Σinv

    AN = A * N
    ŷ, histo = lsmr(AN, b; atol=tol, btol=tol, itmax=itmax, history=true)

    x̂ = N * ŷ
    return x̂, histo
end

lsrn_lsmr (generic function with 1 method)

In [ ]:
@benchmark lsrn_lsmr(A, b; gamma=2.0, tol=1e-10, itmax=2000) 

In [ ]:
BLAS.set_num_threads(1)

In [ ]:
function lsrn_cg(A, b; gamma=2.0, tol=1e-10, itmax=2000)
    m, n = size(A)
    s = ceil(Int, gamma * n)
    G = randn(s, m)
    Ã = G * A

    # SVD compacte
    Ũ, Σ̃, Ṽ = svd(Ã; full=false)
    r = sum(Σ̃ .> 1e-12)
    Σinv = Diagonal(1.0 ./ Σ̃[1:r])
    V_r = Ṽ[:,1:r]
    N = V_r * Σinv

    # Forme normale
    AN = A * N
    AtAN = AN' * AN
    AtbN = AN' * b

    # Résoudre par CG
    ŷ, histo = cg(AtAN, AtbN; atol=tol, rtol=tol, itmax=itmax, history=true)

    # Reconstruire la solution min-norme
    x̂ = N * ŷ
    return x̂, histo
end

In [ ]:
function lsrn_lsqr_parallel(A, b; gamma=2.0, tol=1e-10, itmax=2000)
    m, n = size(A)
    s = ceil(Int, gamma * n)
    G = randn(s, m)

    B = zeros(s, n)

    nthreads = Threads.nthreads()
    blocksize = ceil(Int, s / nthreads)

    Threads.@threads for t = 1:nthreads
        i1 = (t-1)*blocksize + 1
        i2 = min(t*blocksize, s)
        if i1 <= i2 
            B[i1:i2, :] .= G[i1:i2, :] * A
        end
    end

    Ũ, Σ̃, Ṽ = svd(B; full=false)
    r = sum(Σ̃ .> 1e-12)
    Σinv = Diagonal(1.0 ./ Σ̃[1:r])
    V_r = Ṽ[:, 1:r]
    N = V_r * Σinv

    AN = A * N
    ŷ, histo = lsqr(AN, b; atol=tol, btol=tol, itmax=itmax, history=true)

    x̂ = N * ŷ
    return x̂, histo
end